# Latest, working great code for Scene Graph Predict

EXPECTED INPUT
```python
one_segment_CLIP_output = {
    "video_stem": str(Path(video_filepath).stem),
    "segment_id": str(Path(video_filepath).stem) + f"_{i}",
    "segment_index": i,
    "total_segments": len(whisper_segments),
    "segment_total_time": whisper_segments[i]['end'] - whisper_segments[i]['start'],
    "captions": whisper_segments[i]['caption'],
    "segment_start_time": whisper_segments[i]['start'],
    "segment_end_time": whisper_segments[i]['end'],
    "num_frames_per_segment": self.num_frames,
    "frame_embeddings": image_feature,
    "text_caption_embeddings": caption_feature,
    "segment_frames": segment_frame,
    "frame_embeddings_shape": image_feature.shape,          # trying the FLATTEN technique!
    "text_caption_embeddings_shape": caption_feature.shape,
    "segment_frames_shape": segment_frame.shape,
    "scene_graph_captions": "comma separated string (list of scene graph outputs)"  # <--- this is all we're adding
}
```

In [1]:
from faster_OpenPSG.predict import Predictor 
my_pred = Predictor()
my_pred.setup()

/home/kastan/utils/miniconda3/envs/openpsg_custom/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load checkpoint from local path: faster_OpenPSG/checkpoints/epoch_60.pth


In [4]:
import jsonlines 
import cv2
import json_numpy

NUM_RELATIONS = 5
INPUT_FILENAME = '/mnt/storage_ssd/parallel_10_clip_output.jsonl'
# INPUT_FILENAME = '/mnt/storage_ssd/parallel_14_clip_output.jsonl'

with jsonlines.open(INPUT_FILENAME) as reader, jsonlines.open(INPUT_FILENAME, mode='a') as writer:
    sro_list = set()
    for obj_idx, test_sample in enumerate(reader):
        num_frames_counter = 0
        if test_sample['segment_frames'][num_frames_counter] is not None:
            print(test_sample['segment_frames'][num_frames_counter])
            frame = json_numpy.loads(test_sample['segment_frames'][num_frames_counter])
            frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            
            curr_sro = my_pred.predict(image=frame_rgb, num_rel=NUM_RELATIONS)
            sro_list.update(set(curr_sro))
            print(sro_list)
            num_frames_counter += 1
        test_sample['scene_graph_set'] = list(sro_list)
        writer.write(test_sample)
        if obj_idx >= 1:
            break
        break

{"__numpy__": "...", "dtype": "|u1", "shape": [360, 480, 3]}
{'the sky over road', 'the sky over group of buildings', 'the sky over truck', 'truck driving on road', 'truck parked on road'}


# Embed with Flan-T5

In [2]:
# !pip install transformers

  Using cached transformers-4.24.0-py3-none-any.whl (5.5 MB)
  Using cached tokenizers-0.13.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
  Using cached huggingface_hub-0.11.0-py3-none-any.whl (182 kB)


In [1]:
import torch
import accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto", torch_dtype=torch.float16)

/home/kastan/utils/miniconda3/envs/openpsg_custom/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_711558/2934762062.py:6 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_711558/2934762062.py'                       │
│                                                                                                  │
│ /home/kastan/utils/miniconda3/envs/openpsg_custom/lib/python3.7/site-packages/transformers/model │
│ ing_utils.py:1920 in from_pretrained                                                             │
│                                                                                                  │
│   1917 │   │                                                                                     │
│   1918 │   │   if low_cpu_mem_usage:                                                             │
│   1919 │   │   │   # low_cpu_mem_usage requires PyTorch >= 1.9 to have the meta device.          │
│ ❱ 1920 │   │   │   require_version_core("torch>=1.9")                                            │
│   1921 │   │   │                                                                                 │
│   1922 │   │   │   if is_deepspeed_zero3_enabled():                                              │
│   1923 │   │   │   │   raise ValueError(                                                         │
│                                                                                                  │
│ /home/kastan/utils/miniconda3/envs/openpsg_custom/lib/python3.7/site-packages/transformers/utils │
│ /versions.py:123 in require_version_core                                                         │
│                                                                                                  │
│   120 def require_version_core(requirement):                                                     │
│   121 │   """require_version wrapper which emits a core-specific hint on failure"""              │
│   122 │   hint = "Try: pip install transformers -U or pip install -e '.[dev]' if you're workin   │
│ ❱ 123 │   return require_version(requirement, hint)                                              │
│   124                                                                                            │
│                                                                                                  │
│ /home/kastan/utils/miniconda3/envs/openpsg_custom/lib/python3.7/site-packages/transformers/utils │
│ /versions.py:117 in require_version                                                              │
│                                                                                                  │
│   114 │   # check that the right version is installed if version number or a range was provide   │
│   115 │   if want_ver is not None:                                                               │
│   116 │   │   for op, want_ver in wanted.items():                                                │
│ ❱ 117 │   │   │   _compare_versions(op, got_ver, want_ver, requirement, pkg, hint)               │
│   118                                                                                            │
│   119                                                                                            │
│   120 def require_version_core(requirement):                                                     │
│                                                                                                  │
│ /home/kastan/utils/miniconda3/envs/openpsg_custom/lib/python3.7/site-packages/transformers/utils │
│ /versions.py:51 in _compare_versions                                                             │
│                                                                                                  │
│    48 │   │   )                                            

In [7]:
CONTEXT = "{Example: A Two-Bit Gray Code Counter}Let's begin with a two-bit Gray code counter with no inputs.As we mentioned in Notes Set 2.1, a Gray code is a cycle over allbit patterns of a certain length in which consecutive patterns differin exactly one bit.For simplicity, our first few examples are based on counters anduse the internal stateof the FSM as the output values.  You should already knowhow to design combinational logic for the outputs if it were necessary.The inputs to a counter, if any, are typically limited to functionssuch as starting and stopping the counter, controlling the counting direction, and resetting the counter to a particular state.A fully-specified transition diagram for a two-bit Gray code counter appears below.With no inputs, the states simply form a loop, withthe counter moving from one state to the next each cycle.Each state in the diagram is marked with the internal state value S_1S_0 (before the ``/'') and the output Z_1Z_0 (after the ``/''), which are always equal for this counter.Based on the transition diagram, we can fill in the K-maps for the next-state values S_1^+ and S_0^+ as shown to the right of thetransition diagram, then derive algebraic expressions in the usual way to obtainS_1^+=S_0 and S_0^+={{S_1}}.We then use the next-state logic to develop the implementationshown on the far right, completing our first counter design."
PROMPT = "Please answer this person's question accurately, clearly and concicely. Context: "
QUESTION = "Question: What are the inputs and outputs of a Gray code counter? "
input_text = PROMPT + CONTEXT + QUESTION + "Answer: "

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=1024)
print(tokenizer.decode(outputs[0]))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_666952/2019990087.py:6 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_666952/2019990087.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tokenizer' is not defined

In [3]:
# TODO: read in and make sure the first 5 are perfect. 

with jsonlines.open('/mnt/storage_ssd/JOHAN_parallel_10_clip_output.jsonl') as reader:
    for obj_idx, test_sample in enumerate(reader):
        
        print(test_sample['scene_graph_set'])

['truck parked on road', 'the sky over truck', 'truck driving on road', 'the sky over group of buildings', 'the sky over road']
['the sky over truck', 'truck driving on road', 'person walking on the pavement', 'the sky over road', 'person standing on the pavement', 'truck parked on road', 'bench on the pavement', 'person carrying handbag', 'person carrying backpack', 'the sky over group of buildings']
['the sky over truck', 'truck driving on road', 'person walking on the pavement', 'the sky over road', 'person standing on the pavement', 'the floor on the floor', 'person beside window-other', 'truck parked on road', 'bench on the pavement', 'person carrying handbag', 'person standing on the floor', 'person carrying backpack', 'the sky over group of buildings', 'window-other on the floor', 'papers on the floor']
['the sky over truck', 'the sky over the pavement', 'bench on the pavement', 'the sky over group of trees', 'person carrying backpack', 'the sky over group of buildings', 'window